In [ ]:
# Gerekli kütüphaneleri yüklüyoruz (NLTK'dan Türkçe listesini almak için)
!pip install nltk
import nltk
nltk.download('stopwords')

# Gerekli tüm importları yapıyoruz
import pandas as pd
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr
import numpy as np
from nltk.corpus import stopwords
import os

# --- 1. YAPILANDIRMA VE VERİ YÜKLEME ---


FILE_NAME = 'user_stories.xlsx'
# Hangi INVEST skorunu tahmin edeceğimizi seçiyoruz (Valuable)
TARGET_COLUMN = 'INV_3_VALUABLE'

# 1. Dosyayı Yeniden Yükleme
print(f"\nLütfen '{FILE_NAME}' adlı (Excel) dosyanızı seçin.")
uploaded = files.upload()

# 2. Veriyi Oku ve Temizle
try:
    df = pd.read_excel(FILE_NAME)
    df.columns = df.columns.str.upper().str.replace(' ', '_')

    # --- KRİTİK ÇÖZÜM (TypeError: ... not 'datetime.datetime' için) ---
    # Pandas'a, bu sütunları okurken hataları (Tarih formatı gibi) görmezden gelmesini
    # ve zorla SAYI'ya (numeric) çevirmesini söylüyoruz.
    # Tarih formatını sayıya çeviremezse, o hücreyi 'NaN' (Boş) yapar.
    df[TARGET_COLUMN] = pd.to_numeric(df[TARGET_COLUMN], errors='coerce')
    df['TEXT'] = df['TEXT'].astype(str) # Metin sütununu zorla metin yap

    # GÜVENLİK KODU 2 (NaN Hatası Önlemi):
    # 'coerce' (zorlama) işlemi nedeniyle oluşan boş satırları kaldır
    df = df.dropna(subset=[TARGET_COLUMN, 'TEXT'])

    print(f"\nDosya başarıyla yüklendi ve veri tipleri SAYI'ya dönüştürüldü. Kalan satır: {len(df)}")

    # 3. Veriyi Bölme ve TF-IDF
    X = df['TEXT']
    Y = df[TARGET_COLUMN].values.ravel()

    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=0.15, random_state=42
    )

    # 4. TF-IDF (Hata Çözümü)
    print("2. TF-IDF Özellik Mühendisliği Yapılıyor...")
    TURKISH_STOP_WORDS = stopwords.words('turkish')
    vectorizer = TfidfVectorizer(max_features=1000, stop_words=TURKISH_STOP_WORDS)
    X_train_features = vectorizer.fit_transform(X_train)
    X_test_features = vectorizer.transform(X_test)

    # 5. Modeli Eğitme (CPU Kullanımı)
    print("3. Geleneksel ML Modeli (SVR) Eğitiliyor...")
    model_svr = SVR(kernel='rbf', C=1.0)
    model_svr.fit(X_train_features, Y_train)

    # 6. Tahmin ve Metrik Hesaplama
    predictions = model_svr.predict(X_test_features)

    mae = mean_absolute_error(Y_test, predictions)
    rmse = np.sqrt(mean_squared_error(Y_test, predictions))
    pearson_r, _ = pearsonr(Y_test, predictions)

    # --- NİHAİ RAPOR (BÖLÜM 4.2) ---
    print("\n--- Geleneksel ML (SVR) Nihai Bulguları ---")
    print(f"1. Ortalama Mutlak Hata (MAE): {mae:.4f}")
    print(f"2. Hata Karekök Ortalaması (RMSE): {rmse:.4f}")
    print(f"3. Uzman Uyumu (Pearson r): {pearson_r:.4f}")

    print("\n[BAŞARI]: H3 ve H5 hipotezleri için nihai sonuçlar üretildi.")

except Exception as e:
    print(f"\nGENEL HATA: {e}")


Lütfen 'user_stories1.xlsx' adlı (Excel) dosyanızı TEKRAR seçin.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Saving user_stories1.xlsx to user_stories1 (3).xlsx

Dosya başarıyla yüklendi ve veri tipleri zorla SAYI'ya dönüştürüldü. Kalan satır: 26
2. TF-IDF Özellik Mühendisliği Yapılıyor...
3. Geleneksel ML Modeli (SVR) Eğitiliyor...

--- Geleneksel ML (SVR) Nihai Bulguları ---
1. Ortalama Mutlak Hata (MAE): 0.5473
2. Hata Karekök Ortalaması (RMSE): 0.5960
3. Uzman Uyumu (Pearson r): 0.9700

[BAŞARI]: H3 ve H5 hipotezleri için nihai sonuçlar üretildi.
